In [116]:
from collections import defaultdict
import json
import re
import random
import os

In [44]:
recipes = []

bow = defaultdict(int)


counter = 0
with open('./data.json') as json_file:
    data = json.load(json_file)
    for p in data:
        recipes.append(p)
        for word in p['instructions'].lower().split(' '):
            bow[word] +=1


bow = {k: v for k, v in sorted(bow.items(), key=lambda item: item[1], reverse=True)}



In [119]:
# does not have quantities
def crossover_instructions(recipe1, recipe2, ingredients_final):

    # preheating
    regex ='preheat (\w+\s)+(\d+)\sdegrees F'
    result1 = re.search(regex, recipe1['instructions'],  re.IGNORECASE | re.MULTILINE)
    result2 = re.search(regex, recipe2['instructions'],  re.IGNORECASE | re.MULTILINE)

    
    temp1 = -1
    temp2 = -1

    if result1 != None:
        temp1 = int(result1.group(2))
    if result2 != None:
        temp2 = int(result2.group(2))

    if temp1 !=-1 and temp2 !=-1:
        temp = round(random.uniform(temp1, temp2),0)
    elif temp1 != -1:
        temp = temp1
    else:
        temp = temp2

    first_step = "Preheat the oven at " + str(temp) + " F degrees."

    # add fat and sugar
    first = []
    second = []
    third = []

    for ingredient in ingredients_final:
        if ingredient['role'] == 'fat' or ingredient['role'] == 'sugar':
            first.append(ingredient['name'])
        #flour, rising agent, binding agent
        elif ingredient['role'] == 'flour' or ingredient['role'] == 'rising agent' or ingredient['role'] == 'binding agent':
            second.append(ingredient['name'])
        # add flavour texture
        else:
            third.append(ingredient['name'])


    second_step = 'Add ' + ', '.join(first) + '. '
    second_step += 'Then add ' + ', '.join(second) + '. '
    second_step += 'Finally, join the following flavour textures: ' + ', '.join(third) +'. '


    # baking
    lower = 10000
    upper = -1
    regex2 = '((\d+) to)? (\d+) minutes'
    times1 = re.search(regex2, recipe1['instructions'],  re.IGNORECASE | re.MULTILINE)
    if times1 != None:
        #print(times1.groups())
        if times1.group(1) == None:
            lower = min(int(times1.group(3)),lower)
            upper = max(int(times1.group(3)), upper)

        else:
            lower = min(int(times1.group(2)), lower)
            upper = max(int(times1.group(3)), upper)

    times2 = re.search(regex2, recipe2['instructions'],  re.IGNORECASE | re.MULTILINE)

    if times2 != None:
        #print(times2.groups())
        if times2.group(1) == None:
            lower = min(int(times2.group(3)), lower)
            upper = max(int(times2.group(3)), upper)
        else:
            lower = min(int(times2.group(2)), lower)
            upper = max(int(times2.group(3)), upper)



    third_step = "Put the dough in the oven from " + str(lower) + " to " + str(upper) + " minutes. Take it from the oven and let it cool down."
    #print(third_step)
    return first_step + os.linesep + second_step + os.linesep + third_step




print(crossover_instructions(recipes[3], recipes[100], recipes[2]['ingredients']))


Preheat the oven at 350 F degrees.
Add unsalted butter, packed brown sugar, white sugar. Then add all-purpose flour, baking soda, egg, egg yolk. Finally, join the following flavour textures: salt, vanilla extract, semisweet chocolate chips. 
Put the dough in the oven from 10 to 12 minutes. Take it from the oven and let it cool down.
